In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import os
import warnings

warnings.filterwarnings('ignore')

### Load cleaned comments and article data 

In [ ]:
%%time

# Load cleaned comment data
comments = pd.read_csv('../data/cleaned/comments.csv', header=0, parse_dates=['scrape_datetime','comment_datetime_clean'])
comments.shape

In [ ]:
%%time

# Load cleaned article data
articles = pd.read_csv('../data/cleaned/articles.csv', header=0, parse_dates=['scrape_datetime','post_datetime'])
articles.shape

In [ ]:
articles.head(1)

### Add article post datetime, text, author name to comments 

In [ ]:
comments_merge = comments.merge(articles[['article_url','post_datetime','article_body','author']], how='left', on='article_url')

### Filter the data by month and engineer features by month

In [ ]:
# Identify the periods that will make up the number of monthly datasets
comments_merge['post_datetime_year'] = pd.to_datetime(comments_merge['post_datetime']).dt.to_period('Y')
comments_merge.post_datetime_year.min(), comments_merge.post_datetime_year.max()

In [ ]:
%%time

# How many yearly datasets
print(comments_merge.post_datetime_year.nunique(), '\n')

# Iterate through the months to create the datasets
for year in comments_merge.post_datetime_year.unique()[:1]:
    
    # To skip NaT values
    if len(str(year)) != 4:
        continue
    
    # Filter data
    comments = comments_merge[comments_merge.post_datetime_year == year]
    print(year)
    
    # Calculate the total number of comments by each commentor
    commentor_features = pd.DataFrame(comments.groupby(['commentor']).size())
    commentor_features['commentor'] = commentor_features.index
    commentor_features.columns = ['total_number_of_comments','commentor']
    commentor_features.to_csv(f'../data/cluster_features_yearly/01_total_number_of_comments_{year}.csv', header=True, index=True)

    # Calculate the number of unique articles the commentor commented on
    unique_articles = comments.groupby(['commentor','article_url']).size()
    unique_articles_df = pd.DataFrame(unique_articles.groupby(['commentor']).size())
    unique_articles_df.columns = ['number_of_articles_commented_on']
    unique_articles_df.to_csv(f'../data/cluster_features_yearly/02_unique_articles_df_{year}.csv', header=True, index=True)
    # unique_articles_df.head()
    
    # Calculate the number of unique articles the commentor commented on exactly once
    unique_articles_single_comment = comments.groupby(['commentor','article_url']).size()
    unique_articles_single_comment_df = pd.DataFrame(unique_articles_single_comment)
    unique_articles_single_comment_df = unique_articles_single_comment_df[unique_articles_single_comment_df[0] == 1]
    unique_articles_single_comment_df = pd.DataFrame(unique_articles_single_comment_df.groupby(['commentor']).size())
    unique_articles_single_comment_df.columns = ['number_of_articles_w_exactly_one_comment']
    unique_articles_single_comment_df.to_csv(f'../data/cluster_features_yearly/03_unique_articles_single_comment_df_{year}.csv', header=True, index=True)
    # unique_articles_single_comment_df.head()
    
    # Calculate the number of unique articles the commentor commented on more than once
    unique_articles_mulitple_comment = comments.groupby(['commentor','article_url']).size()
    unique_articles_mulitple_comment_df = pd.DataFrame(unique_articles_mulitple_comment)
    unique_articles_mulitple_comment_df = unique_articles_mulitple_comment_df[unique_articles_mulitple_comment_df[0] > 1]
    unique_articles_mulitple_comment_df = pd.DataFrame(unique_articles_mulitple_comment_df.groupby(['commentor']).size())
    unique_articles_mulitple_comment_df.columns = ['number_of_articles_w_more_than_one_comment']
    unique_articles_mulitple_comment_df.to_csv(f'../data/cluster_features_yearly/04_unique_articles_mulitple_comment_df_{year}.csv', header=True, index=True)
    # unique_articles_mulitple_comment_df.head()
    
    # Calculate how long (in days) a commentor has been active on pft
    commentor_activity_duration = comments.groupby(['commentor']).agg({'comment_datetime_clean':['min','max']})
    commentor_activity_duration.columns = commentor_activity_duration.columns.droplevel()
    commentor_activity_duration['commentor_activity_duration_in_days'] = (commentor_activity_duration['max'] - commentor_activity_duration['min']).dt.days
    commentor_activity_duration.to_csv(f'../data/cluster_features_yearly/05_commentor_activity_duration_{year}.csv', header=True, index=True)
    # commentor_activity_duration.head()
    
    # Calcualte the length of the commentor's username
    commentor_username_length = comments.groupby(['commentor']).size()
    commentor_username_length = pd.DataFrame(commentor_username_length)
    commentor_username_length['username'] = commentor_username_length.index
    commentor_username_length['username_length'] = commentor_username_length['username'].str.len()
    commentor_username_length.drop([0], axis=1, inplace=True)

    # Calculate the number of letters, numbers, and spaces in the commentor's username
    commentor_username_length['username_alpha_chars'] = commentor_username_length['username'].apply(lambda username: sum(x.isalpha() for x in username))
    commentor_username_length['username_numeric_chars'] = commentor_username_length['username'].apply(lambda username: sum(x.isdigit() for x in username))
    commentor_username_length['username_space_chars'] = commentor_username_length['username'].apply(lambda username: sum(x.isspace() for x in username))
    commentor_username_length.to_csv(f'../data/cluster_features_yearly/06_commentor_username_length_{year}.csv', header=True, index=True)
    # commentor_username_length.head()
    
    # Calculate the mean, median, min and max length of comments (characters)
    comments['comment_body_length'] = comments['comment_body'].str.len()
    commentor_comment_body_metrics = comments.groupby(['commentor']).agg({'comment_body_length':['mean','median','min','max','sum']})
    commentor_comment_body_metrics.columns = commentor_comment_body_metrics.columns.droplevel()
    commentor_comment_body_metrics.columns = ['comment_length_mean','comment_length_median','comment_length_min','comment_length_max','comment_length_total']
    commentor_comment_body_metrics.to_csv(f'../data/cluster_features_yearly/07_commentor_comment_body_metrics_{year}.csv', header=True, index=True)
    # commentor_comment_body_metrics.head()
    
    # Calculate the average, median, min, max hours between when article was published and comment was made
    articles_w_dates = articles.drop_duplicates(subset=['article_url','post_datetime'])
    comments_between = pd.merge(comments, articles_w_dates[['article_url','post_datetime']], how='left', on='article_url')
    comments_between = comments_between[(comments_between.comment_datetime_clean >= comments_between.post_datetime_x)]
    comments_between['hours_btween'] = (comments_between.comment_datetime_clean - comments_between.post_datetime_x) / pd.Timedelta(hours=1)

    hours_between_metrics = comments_between.groupby(['commentor']).agg({'hours_btween':['mean','median','min','max']})
    hours_between_metrics.columns = hours_between_metrics.columns.droplevel()
    hours_between_metrics.columns = ['hours_between_mean','hours_between_median','hours_between_min','hours_between_max']
    hours_between_metrics.to_csv(f'../data/cluster_features_yearly/08_hours_between_metrics_{year}.csv', header=True, index=True)
    # hours_between_metrics.head()
    
    # Calculate which days of the week comments were made on
    comments['comment_date_dow'] = comments['comment_datetime_clean'].dt.day_name()
    comments_dow = pd.pivot_table(comments[['article_url','commentor','comment_date_dow']], index=['commentor'],
                        columns=['comment_date_dow'], aggfunc='count', fill_value=0)
    comments_dow.columns = comments_dow.columns.droplevel()
    comments_dow.columns = ['comments_on_' + c for c in comments_dow.columns]
    comments_dow.to_csv(f'../data/cluster_features_yearly/09_comments_dow_{year}.csv', header=True, index=True)
    # comments_dow.head()
    
    # Calculate which hours of the day comments were made on
    comments['comment_date_hour'] = comments['comment_datetime_clean'].dt.hour
    comments_hour = pd.pivot_table(comments[['article_url','commentor','comment_date_hour']], index=['commentor'],
                        columns=['comment_date_hour'], aggfunc='count', fill_value=0)
    comments_hour.columns = comments_hour.columns.droplevel()
    comments_hour.columns = ['comments_on_hour_' + str(c) for c in comments_hour.columns]
    comments_hour.to_csv(f'../data/cluster_features_yearly/10_comments_hour_{year}.csv', header=True, index=True)
    # comments_hour.head()
    
    # Count number of comment made "in-season" vs "off-season"
    # In-season being between 9/1 and 2/1, inclusive
    comments['comment_date_month'] = comments['comment_datetime_clean'].dt.month
    comments['in_season_flag'] = np.where((comments['comment_date_month'] >= 9) | (comments['comment_date_month'] <= 2), 1, 0)
    in_season_comments = comments.groupby(['commentor']).agg({'in_season_flag':['count','sum']})
    in_season_comments.columns = in_season_comments.columns.droplevel()
    in_season_comments.columns = ['total_comments','number_in_season_comments']
    in_season_comments['number_out_season_comments'] = in_season_comments['total_comments'] - in_season_comments['number_in_season_comments']
    in_season_comments.to_csv(f'../data/cluster_features_yearly/11_in_season_comments_{year}.csv', header=True, index=True)
    # in_season_comments.head()
    
    # Calculate the most number of comments each commentor posted in a single day
    commentor_max_comments = pd.DataFrame(comments.groupby(['commentor','comment_datetime_clean']).size()).reset_index()
    commentor_max_comments_df = pd.DataFrame(commentor_max_comments.groupby(['commentor'])[0].max())
    commentor_max_comments_df.columns = ['max_number_comments_in_single_day']
    commentor_max_comments_df.to_csv(f'../data/cluster_features_yearly/12_commentor_max_comments_df_{year}.csv', header=True, index=True)
    # commentor_max_comments_df.head()
    

In [ ]:
comments.columns

In [ ]:
%%time

# Calculate the number of comments made by commentors on articles written about each NFL team

# Read in nfl teams
with open('../data/reference/nfl_team_names.txt') as f:
    content = f.readlines()

# Create DataFrame of unique articles (added for efficiency)
articles_yearly = pd.DataFrame(comments['article_body'])
articles_yearly_unique = articles_yearly.drop_duplicates()

# Flag articles by which teams they mention
nfl_teams = [x.strip() for x in content]
for team in nfl_teams:
    articles_yearly_unique[f'article_mentions_{team}'] = articles_yearly_unique['article_body'].str.lower().str.contains(f'{team}')

# Combine comments with articles flagged by team and fix redskins/WFT name change
comments_with_teams = pd.merge(comments, articles_yearly_unique, how='left', on='article_body')
comments_with_teams['article_mentions_football_team'] = comments_with_teams['article_mentions_football team'] + comments_with_teams['article_mentions_redskins']
comments_with_teams.drop(labels=['article_mentions_redskins','article_mentions_football team'], axis=1, inplace=True)

# Aggregate number of articles commented on by 
cols_to_sum = [c for c in comments_with_teams if 'article_mentions_' in c]
commentor_by_articles_mentioning_team = comments_with_teams.groupby(['commentor'])[cols_to_sum].apply(lambda x : x.astype(int).sum())
commentor_by_articles_mentioning_team.head()

In [ ]:
%%time

# Calculate the number of comments made by commentors on articles about each player position (e.g, QB, QB, TE)
player_positions_full = ['quarterback','wide receiver','tackle','guard','center','tight end','fullback','halfback','running back',
                    'cornerback','linebacker','defensive end','safety', 'kicker']

# https://help.yahoo.com/kb/position-abbreviations-eligibility-players-play-positions-sln6500.html
# player_positions_abbreviation = ['QB','WR','OL','DL','TE','FB','RB','CB','LB','DE','S','ST']   # can we include position abbreivations that are only one char: S, K, T
# positions = player_positions_full + player_positions_abbreviation

# Create DataFrame of unique articles (added for efficiency)
articles_yearly = pd.DataFrame(comments['article_body'])
articles_yearly_unique = articles_yearly.drop_duplicates()

for position in player_positions_full:
    articles_yearly_unique[f'article_mentions_{position}'] = articles_yearly_unique['article_body'].str.lower().str.contains(f'{position}')

# Combine comments with articles flagged by team and fix redskins/WFT name change
comments_with_positions = pd.merge(comments, articles_yearly_unique, how='left', on='article_body')

# Aggregate number of articles commented on by 
cols_to_sum = [c for c in comments_with_positions if 'article_mentions_' in c]
commentor_by_articles_mentioning_position = comments_with_positions.groupby(['commentor'])[cols_to_sum].apply(lambda x : x.astype(int).sum())
commentor_by_articles_mentioning_position.head()

In [ ]:
# Calculate the number of comments made by commentors on articles about "offense" vs. "defense"
comments['article_mentions_offense'] = comments['article_body'].str.lower().str.contains('offense')
comments['article_mentions_offense'] = comments['article_body'].str.lower().str.contains('defense')

In [ ]:
# Calculate the number of comments made by commentors on articles that mention "fantasy"
comments['article_mentions_offense'] = comments['article_body'].str.lower().str.contains('fantasy')

In [ ]:
# Calculate the number of comments made by commentors on articles that mention "injured","injury", "injuries","questionable"


In [ ]:
# Calculate the number of comments made by commentors on articles that mention "contract"


### Features to add

In [ ]:
# Calculate the number of comments made by commentors on articles about "offense" vs. "defense"
# Calculate the number of comments made by commentors on articles that mention "fantasy"
# Calculate the number of comments made by commentors on articles that mention "injured","injury", "injuries","questionable"
# Calculate the number of comments made by commentors on articles that mention "contract"

# Calculate the number of comments made by commentors on articles written by each author
# Calculate the number of comments by sentiment: {polarity:[positive, negative, neutral], subjectivity:[0,1]}
# Calculate the number of words in the commentor's posts in ALL CAPS
# Calculate readability metrics -> https://github.com/cdimascio/py-readability-metrics



### Load features and combine into a single dataset

In [ ]:
# Inspect files
feature_files = os.listdir('../data/cluster_features_yearly/')
print(feature_files[:13])

In [ ]:
# Validate there are 12 files for all 13 years
for year in range(2008,2021):
    yearly_files = [f for f in feature_files if str(year) in f]
    print(f'Found {len(yearly_files)} feature files for year: {year}')

In [ ]:
# Expected number of features
total_cols = 0
for file in yearly_files:
    df = pd.read_csv(f'../data/cluster_features_yearly/{file}', nrows=5)
    total_cols += df.shape[1] - 1  # Exclude index
total_cols    

In [ ]:
# Iterate through the years
for year in range(2008,2021):
    yearly_files = [f for f in feature_files if str(year) in f]

    # Add the path to each feature file
    feature_files_w_path = list()
    for f in yearly_files:
        feature_files_w_path.append(f'../data/cluster_features_yearly/{f}')

    # Read the first file to use as seed
    df = pd.read_csv(feature_files_w_path[0], header=0, index_col=0)

    # Iterate through files and merge into seed
    for df_ in feature_files_w_path[1:]:
        df_next = pd.read_csv(df_, header=0, index_col=0)
        df = df.merge(df_next, on='commentor', how='left')

    print(year, df.shape)
    
    # Write combined dataset to flat file
    df.to_csv(f'../data/cleaned/yearly_comment_features/comment_features_{year}.csv', header=True, index=True)